# Humans by group
Author: Martin Urbanec <martin.urbanec@wikimedia.cz>

This notebook is designed to estimate the number of humans who are in a particular user group. It can only run in the internal analytics cluster, as it requires access to unredacted `user` table, as that is where system-ness of an account can be easily determined.

The "is human" estimation is done by:

* ignoring known system accounts
* ignoring bot accounts

This is based on [a report made in April 2021](https://people.wikimedia.org/~urbanecm/growth-team/sysops-per-wiki-20210429.html) for the Growth team.

In [1]:
from wmfdata import mariadb, utils
import pandas as pd

pd.set_option('display.max_rows', None)

In [2]:
dfs = []

private = utils.get_dblist('private')
closed = utils.get_dblist('closed')
for wiki in utils.get_dblist('all'):
    if wiki in private or wiki in closed:
        continue
    
    if wiki == 'labtestwiki':
        continue
    
    try:
        dfs.append(mariadb.run('''
        SELECT
            DATABASE() AS wiki,
            ug_expiry IS NULL AS permanent,
            ug_group,
            COUNT(*) AS users
        FROM user_groups
        -- we need the user table, to be able to exclude system accounts
        JOIN user ON user_id=ug_user
        WHERE
            -- Ignore system accounts
            user_token NOT LIKE "%INVALID%" AND

            -- Ignore bots
            ug_user NOT IN (
                SELECT ug_user FROM user_groups WHERE ug_group="bot"
            )
        GROUP BY
            ug_expiry IS NULL,
            ug_group
        ''', wiki))
    except ValueError:
        pass

df = pd.concat(dfs).reset_index(drop=True).fillna(0)

In [3]:
dfPivot = df.loc[df.permanent==1][['wiki', 'ug_group', 'users']].pivot_table(index='wiki', columns=['ug_group'], values='users', fill_value=0)
dfPivot.to_json('/home/urbanecm/Documents/steward/2021-wikimedia-humans-per-group/data/permanent-users-per-group.json')

In [4]:
dfPivot = df.loc[df.permanent==0][['wiki', 'ug_group', 'users']].pivot_table(index='wiki', columns=['ug_group'], values='users', fill_value=0)
dfPivot.to_json('/home/urbanecm/Documents/steward/2021-wikimedia-humans-per-group/data/temporary-users-per-group.json')

In [5]:
dfPivot = df[['wiki', 'ug_group', 'users']].groupby(['wiki', 'ug_group']).sum().pivot_table(index='wiki', columns=['ug_group'], values='users', fill_value=0)
dfPivot.to_json('/home/urbanecm/Documents/steward/2021-wikimedia-humans-per-group/data/total-users-per-group.json')